In [279]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data
hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
temp=pd.concat([hard_womens,hard_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()

In [280]:
import pandas as pd
calc=temp.copy()
# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Keep only the rows where there was a win
player_wins = player_results[player_results['Win'] == True].copy()

# Sort player wins by Player and Date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Function to count wins in the next 30 days after the match (excludes current match)
def wins_last_30_after_match(group):
    result = []
    for i in range(len(group)):
        cutoff = group.iloc[i]['Date']
        # Filter for wins that happened after the cutoff date within the next 30 days
        future_wins = group[(group['Date'] > cutoff) & (group['Date'] <= cutoff + pd.Timedelta(days=30))]
        result.append(len(future_wins))  # Count the wins in the next 30 days
    return pd.Series(result, index=group.index)

# Calculate wins in the next 30 days after each match
player_wins['Wins_Last_30_Days_After'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_after_match)
)

# Merge the calculated win counts back into the original dataframe

# Sort for merge_asof (allows for the "after match" logic)
calc = calc.sort_values('Date')
player_wins = player_wins.sort_values('Date')

# Merge back for Fav (after the match date)
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_After']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_After': 'Fav_Wins_Last_30_Days_After'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='forward'
).drop(columns=['Match_Date'])

# Merge back for Dog (after the match date)
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_After']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_After': 'Dog_Wins_Last_30_Days_After'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='forward'
).drop(columns=['Match_Date'])

# Fill any missing win counts with 0 (if no future wins)
calc['Fav_Wins_Last_30_Days_After'] = calc['Fav_Wins_Last_30_Days_After'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_After'] = calc['Dog_Wins_Last_30_Days_After'].fillna(0).astype(int)

Fav_Win_Count=calc[['Date','Fav','Fav_Wins_Last_30_Days_After']].copy()
Fav_Win_Count['Date'] = pd.to_datetime(Fav_Win_Count['Date'])
latest_per_fav = Fav_Win_Count[Fav_Win_Count['Date'] == Fav_Win_Count.groupby('Fav')['Date'].transform('max')]
latest_per_fav[['Fav','Fav_Wins_Last_30_Days_After']].to_csv('Fav_Win_Count_Hard.csv')
Dog_Win_Count=calc[['Date','Dog','Dog_Wins_Last_30_Days_After']].copy()
Dog_Win_Count['Date'] = pd.to_datetime(Dog_Win_Count['Date'])
latest_per_dog = Dog_Win_Count[Dog_Win_Count['Date'] == Dog_Win_Count.groupby('Dog')['Date'].transform('max')]
latest_per_dog[['Dog','Dog_Wins_Last_30_Days_After']].to_csv('Dog_Win_Count_Hard.csv')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12200\2330817413.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_after_match)


In [281]:
calc=temp.copy()
import pandas as pd

# Ensure 'Date' is a datetime type
calc['Date'] = pd.to_datetime(calc['Date'])

# Melt Fav and Dog results into long format for players and their wins
fav_df = calc[['Date', 'Fav', 'Fav_Win']].rename(columns={'Fav': 'Player', 'Fav_Win': 'Win'})
dog_df = calc[['Date', 'Dog', 'Dog_Win']].rename(columns={'Dog': 'Player', 'Dog_Win': 'Win'})

# Concatenate both datasets
player_results = pd.concat([fav_df, dog_df], ignore_index=True)

# Keep only the rows where there was a win
player_wins = player_results[player_results['Win'] == True].copy()

# Sort player wins by Player and Date
player_wins = player_wins.sort_values(['Player', 'Date'])

# Function to count wins in the last 30 days, excluding the current match
def wins_last_30_before_match(group):
    result = []
    for i in range(len(group)):
        cutoff = group.iloc[i]['Date']
        # Filter for wins that happened before the cutoff date within the last 30 days
        past_wins = group[(group['Date'] < cutoff) & (group['Date'] >= cutoff - pd.Timedelta(days=30))]
        result.append(len(past_wins))  # Count the wins in the past 30 days
    return pd.Series(result, index=group.index)

# Calculate wins in the last 30 days for each player before the match
player_wins['Wins_Last_30_Days_Before'] = (
    player_wins.groupby('Player', group_keys=False)
    .apply(wins_last_30_before_match)
)

# Merge the calculated win counts back into the original dataframe

# Sort for merge_asof (allows for the "before match" logic)
calc = calc.sort_values('Date')
player_wins = player_wins.sort_values('Date')

# Merge back for Fav (before the match date)
fav_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Fav',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Fav_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    fav_win_counts.sort_values('Match_Date'),
    by='Fav',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Merge back for Dog (before the match date)
dog_win_counts = player_wins[['Player', 'Date', 'Wins_Last_30_Days_Before']].rename(columns={
    'Player': 'Dog',
    'Date': 'Match_Date',
    'Wins_Last_30_Days_Before': 'Dog_Wins_Last_30_Days_Before'
})

calc = pd.merge_asof(
    calc.sort_values('Date'),
    dog_win_counts.sort_values('Match_Date'),
    by='Dog',
    left_on='Date',
    right_on='Match_Date',
    direction='backward'
).drop(columns=['Match_Date'])

# Fill any missing win counts with 0 (if no prior wins)
calc['Fav_Wins_Last_30_Days_Before'] = calc['Fav_Wins_Last_30_Days_Before'].fillna(0).astype(int)
calc['Dog_Wins_Last_30_Days_Before'] = calc['Dog_Wins_Last_30_Days_Before'].fillna(0).astype(int)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12200\74366135.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(wins_last_30_before_match)


In [282]:
stake=100
date_filter=calc[calc['Date']>'2024-07-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.5].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
oddsfilter.to_csv('_HardResults_Womens.csv')

Total Profit: 4986.0 300 0.5366666666666666


In [283]:
stake=100
date_filter=calc[calc['Date']>'2024-07-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.7].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Wins_Last_30_Days_Before']>oddsfilter['Fav_Wins_Last_30_Days_Before']]
oddsfilter=oddsfilter[oddsfilter['Fav_Wins_Last_30_Days_Before']==0]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter[['Date','Dog_Odds','Winner','Dog','Fav','Dog_Win_Count','Fav_Loss_Count','Dog_Wins_Last_30_Days_Before','Fav_Wins_Last_30_Days_Before']]
oddsfilter.to_csv('_HardResults_Mens.csv')

Total Profit: 1226.0 128 0.5546875


In [284]:
hardmen=pd.read_csv(r'.\_HardResults_Mens.csv')
hardwommen=pd.read_csv(r'.\_HardResults_Womens.csv')
claymen=pd.read_csv(r'.\_ClayResults_Mens.csv')
claywommen=pd.read_csv(r'.\_ClayResults_Womens.csv')

combined=pd.concat([hardmen,hardwommen,claymen,claywommen])
#combined=pd.concat([claywommen])

import pandas as pd
oddsfilter=combined.copy()

# Ensure date is datetime and sorted
oddsfilter['Date'] = pd.to_datetime(oddsfilter['Date'])
oddsfilter = oddsfilter.sort_values('Date').reset_index(drop=True)

# Initialize bankroll tracking
initial_bankroll = 1000
daily_bankrolls = {}
current_bankroll = initial_bankroll

stakes = []
profits = []
bankrolls = []

# Group by date
for date, day_df in oddsfilter.groupby('Date'):
    stake = 0.05 * current_bankroll  # stake same for all matches on this day

    day_profits = []

    # Compute profit for each row in the day
    for _, row in day_df.iterrows():
        if row['Dog_Win'] == 1:
            profit = row['Dog_Odds'] * stake - stake
        else:
            profit = -stake
        day_profits.append(profit)

        # Append values for each row
        stakes.append(stake)
        profits.append(profit)
        bankrolls.append(current_bankroll)

    # Update bankroll after all bets for the day
    current_bankroll += sum(day_profits)

# Assign columns
oddsfilter['Stake'] = stakes
oddsfilter['Profit'] = profits
oddsfilter['Bankroll'] = bankrolls
oddsfilter[['Date','Dog','Fav','Winner','Dog_Odds','Stake','Profit','Bankroll']].to_clipboard(index=False)


In [285]:
date_filter = pd.read_pickle(r'.\Hard_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_win = pd.read_csv(r'.\Fav_Win_Count_Hard.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count_Hard.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_win_subset = fav_win[['Fav', 'Fav_Wins_Last_30_Days_After']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Wins_Last_30_Days_After']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_win_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
today=date_filter.copy()
today=today[today['Fav_Odds']>1.5]
today=today[today['Dog_Wins_Last_30_Days_After']>today['Fav_Wins_Last_30_Days_After']]
womens=today[today['Sex']=='Womens']
womens

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Wins_Last_30_Days_After,Dog_Wins_Last_30_Days_After


In [286]:
lowodds=pd.read_pickle(r'.\Hard_Today')
lowodds=lowodds[lowodds['Fav_Odds']<1.3].copy()
lowodds=lowodds[lowodds['Fav_Odds']>1]
favs=pd.read_csv('Fav_Hard.csv')
fav_list=favs['Name'].tolist()
lowodds=lowodds[(lowodds['Fav']).isin(fav_list)]
lowodds=lowodds[lowodds['dog_rank']>50]
lowodds

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count


In [ ]:
date_filter = pd.read_pickle(r'.\Hard_Today').drop(columns=['Fav_Loss_Count','Dog_Win_Count'])
fav_win = pd.read_csv(r'.\Fav_Win_Count_Hard.csv')
dog_win = pd.read_csv(r'.\Dog_Win_Count_Hard.csv')

# Keep only necessary columns to avoid duplicates or unnecessary data
fav_win_subset = fav_win[['Fav', 'Fav_Wins_Last_30_Days_After']].drop_duplicates(subset='Fav')
dog_win_subset = dog_win[['Dog', 'Dog_Wins_Last_30_Days_After']].drop_duplicates(subset='Dog')

# Merge based only on player names
date_filter = date_filter.merge(fav_win_subset, on='Fav', how='left')
date_filter = date_filter.merge(dog_win_subset, on='Dog', how='left')
today=date_filter.copy()
today=today[today['Fav_Odds']>1.7]
today=today[today['Dog_Wins_Last_30_Days_After']>today['Fav_Wins_Last_30_Days_After']]
mens=today[today['Sex']!='Womens']
mens

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Wins_Last_30_Days_After,Dog_Wins_Last_30_Days_After
0,03:15,1.53,2.41,Patrick Kypson,Patrick Kypson,92%,13,Rei Sakamoto,50%,6,...,0.4,0.2,0.0,0.0,NaN,NaN,1.0,0.4,0,0


In [288]:
# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")
mens['Date']=date_string
mens.to_sql('Preditions_Hard',con=devengine,if_exists='append')
womens['Date']=date_string
womens.to_sql('Preditions_Hard',con=devengine,if_exists='append')

0

In [289]:
predictions=pd.read_sql_table('Preditions_Hard',con=devengine)
stake=100
results=temp
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],  # Select only the Winner column from results
    on=['Date', 'Fav', 'Dog'], 
    how='left'  # Use left join to retain all rows from predictions
)
#merged_df_filtered=merged_df[merged_df['Dog_Win_Count']>2].copy()
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Loss_Count']>2]
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Odds']>1.7]
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Date']>'2025-07-05']
merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered


ZeroDivisionError: division by zero